In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, logm, LinAlgError
import itertools
import time
from collections import defaultdict, Counter
import warnings
import os
from mpl_toolkits.mplot3d import Axes3D

# --- Configuration ---
GENERATE_FULL_PLOTS = False
NUM_SAMPLES_PER_AK = 9
PATTERN_ROUND_PRECISION = 6

DEBUG_RT_CALCULATION_MAIN = True
MAX_PATHS_TO_DEBUG_RT_MAIN = 3
# --- End Configuration ---

start_time = time.time()
print(f"Script started at: {time.ctime(start_time)}")
print(f"GENERATE_FULL_PLOTS: {GENERATE_FULL_PLOTS}")
print(f"NUM_SAMPLES_PER_AK for P-label pattern: {NUM_SAMPLES_PER_AK}")
print(f"PATTERN_ROUND_PRECISION: {PATTERN_ROUND_PRECISION}")

# --- 1. Definitions and Constants ---
omega = np.exp(2j * np.pi / 3)
dim = 3
sqrt3 = np.sqrt(3)

phase_points_tuples = []
for p_idx_const_setup_final in range(dim): # Renamed
    for q_idx_const_setup_final in range(dim): phase_points_tuples.append((p_idx_const_setup_final, q_idx_const_setup_final))

Id = np.identity(dim, dtype=complex)

lambda_1=np.array([[0,1,0],[1,0,0],[0,0,0]],dtype=complex); lambda_2=np.array([[0,-1j,0],[1j,0,0],[0,0,0]],dtype=complex)
lambda_3=np.array([[1,0,0],[0,-1,0],[0,0,0]],dtype=complex); lambda_4=np.array([[0,0,1],[0,0,0],[1,0,0]],dtype=complex)
lambda_5=np.array([[0,0,-1j],[0,0,0],[1j,0,0]],dtype=complex); lambda_6=np.array([[0,0,0],[0,0,1],[0,1,0]],dtype=complex)
lambda_7=np.array([[0,0,0],[0,0,-1j],[0,1j,0]],dtype=complex); lambda_8=(1/sqrt3)*np.array([[1,0,0],[0,1,0],[0,0,-2]],dtype=complex)
lambdas = {1:lambda_1, 2:lambda_2, 3:lambda_3, 4:lambda_4, 5:lambda_5, 6:lambda_6, 7:lambda_7, 8:lambda_8}
lambda_names = {1:r'$\lambda_1$', 2:r'$\lambda_2$', 3:r'$\lambda_3$', 4:r'$\lambda_4$', 5:r'$\lambda_5$', 6:r'$\lambda_6$', 7:r'$\lambda_7$', 8:r'$\lambda_8$'}

SU2_146_INDICES = [1, 4, 6]; SU2_257_INDICES = [2, 5, 7]; DIAGONAL_INDICES = [3, 8]
ALL_LEAKAGE_INDICES = [1, 2, 4, 5, 6, 7]

leakage_channel_colors = {1:'tab:blue',2:'tab:orange',4:'tab:green',5:'tab:red',6:'tab:purple',7:'tab:brown'}
code_space_color = 'tab:gray'; COLOR_146 = 'blue'; COLOR_257 = 'red'

A_ops_num={}; D_ops_num={}
D_matrices_new = {
    (0,0):Id,(0,1):np.array([[0,0,1],[1,0,0],[0,1,0]],dtype=complex),(0,2):np.array([[0,1,0],[0,0,1],[1,0,0]],dtype=complex),
    (1,0):np.array([[1,0,0],[0,omega,0],[0,0,omega**2]],dtype=complex),(1,1):np.array([[0,0,omega],[omega**2,0,0],[0,1,0]],dtype=complex),
    (1,2):np.array([[0,omega**2,0],[0,0,1],[omega,0,0]],dtype=complex),(2,0):np.array([[1,0,0],[0,omega**2,0],[0,0,omega]],dtype=complex),
    (2,1):np.array([[0,0,omega**2],[omega,0,0],[0,1,0]],dtype=complex),(2,2):np.array([[0,omega,0],[0,0,1],[omega**2,0,0]],dtype=complex)
}
A_matrices_new = {
    (0,0):np.array([[1,0,0],[0,0,1],[0,1,0]],dtype=complex),(0,1):np.array([[0,0,1],[0,1,0],[1,0,0]],dtype=complex),
    (0,2):np.array([[0,1,0],[1,0,0],[0,0,1]],dtype=complex),(1,0):np.array([[1,0,0],[0,0,omega**2],[0,omega,0]],dtype=complex),
    (1,1):np.array([[0,0,omega],[0,1,0],[omega**2,0,0]],dtype=complex),(1,2):np.array([[0,omega**2,0],[omega,0,0],[0,0,1]],dtype=complex),
    (2,0):np.array([[1,0,0],[0,0,omega],[0,omega**2,0]],dtype=complex),(2,1):np.array([[0,0,omega**2],[0,1,0],[omega,0,0]],dtype=complex),
    (2,2):np.array([[0,omega,0],[omega**2,0,0],[0,0,1]],dtype=complex)
}
print("\n--- Populating Operators ---")
for m_op_idx_s_final,n_op_idx_s_final in D_matrices_new: D_ops_num[(m_op_idx_s_final,n_op_idx_s_final)]=D_matrices_new[(m_op_idx_s_final,n_op_idx_s_final)]
for p_op_idx_s_final,q_op_idx_s_final in A_matrices_new:
    A_pq_local_s_final=A_matrices_new[(p_op_idx_s_final,q_op_idx_s_final)]; assert np.allclose(A_pq_local_s_final,A_pq_local_s_final.conj().T,atol=1e-9),f"A({p_op_idx_s_final},{q_op_idx_s_final}) HermFail!"
    A_ops_num[(p_op_idx_s_final,q_op_idx_s_final)]=A_pq_local_s_final
print("Done populating A(p,q) and D(m,n).")

# --- Direct Operator Identity Check ---
print("\n\n=== Verifying Operator Identity D A D_dag = A_transformed (Matrix Level) ===")
total_op_id_checks_final = 0
successful_op_id_checks_final = 0
for dp_op_id_check_final, dq_op_id_check_final in D_ops_num.keys():
    D_op_check_final = D_ops_num[(dp_op_id_check_final, dq_op_id_check_final)]
    for p_start_op_id_check_final, q_start_op_id_check_final in A_ops_num.keys():
        A_start_op_id_check_final = A_ops_num[(p_start_op_id_check_final, q_start_op_id_check_final)]
        LHS_op_id_check_final = D_op_check_final @ A_start_op_id_check_final @ D_op_check_final.conj().T
        p_new_op_id_check_final = (dp_op_id_check_final + p_start_op_id_check_final) % dim
        q_new_op_id_check_final = (dq_op_id_check_final + q_start_op_id_check_final) % dim
        RHS_op_id_check_final = A_ops_num[(p_new_op_id_check_final, q_new_op_id_check_final)]
        total_op_id_checks_final += 1
        if np.allclose(LHS_op_id_check_final, RHS_op_id_check_final, atol=1e-9):
            successful_op_id_checks_final += 1
        else:
            print(f"  MISMATCH! For D({dp_op_id_check_final},{dq_op_id_check_final}) and A({p_start_op_id_check_final},{q_start_op_id_check_final}): -> Expected A({p_new_op_id_check_final},{q_new_op_id_check_final})")
            print(f"    Max element diff: {np.max(np.abs(LHS_op_id_check_final-RHS_op_id_check_final)):.2e}")
print(f"Operator Identity Check Complete: {successful_op_id_checks_final}/{total_op_id_checks_final} transformations verified.")
if successful_op_id_checks_final != total_op_id_checks_final:
    print("  WARNING: Some direct operator identity checks FAILED!")


def get_gellmann_coeffs(matrix, lambdas_dict_param):
    coeffs = np.zeros(8, dtype=complex)
    for k_idx_gc_f_final in range(1, 9): coeffs[k_idx_gc_f_final-1] = 0.5 * np.trace(matrix @ lambdas_dict_param[k_idx_gc_f_final])
    return coeffs

print("\n--- Pre-calculating Static Gell-Mann Coefficients for A_pq operators ---")
static_A_pq_coeffs = {}
for p_s_static_c_final in range(dim):
    for q_s_static_c_final in range(dim):
        pq_tuple_s_c_final = (p_s_static_c_final, q_s_static_c_final)
        A_operator_s_c_final = A_ops_num[pq_tuple_s_c_final]
        coeffs_vec_s_c_final = get_gellmann_coeffs(A_operator_s_c_final, lambdas).real
        static_A_pq_coeffs[pq_tuple_s_c_final] = tuple(np.round(coeffs_vec_s_c_final, PATTERN_ROUND_PRECISION))
print(f"Done pre-calculating static coefficients for {len(static_A_pq_coeffs)} A_pq operators.")

def calculate_path_coeffs_all_lambdas(start_pq_tuple, end_pq_tuple, lambdas_dict_param, num_steps_param=100):
    A_start_func = A_ops_num[start_pq_tuple]
    dp_func = (end_pq_tuple[0] - start_pq_tuple[0] + dim) % dim
    dq_func = (end_pq_tuple[1] - start_pq_tuple[1] + dim) % dim
    D_eff_tuple_func = (dp_func, dq_func)
    D_disp_func = D_ops_num[D_eff_tuple_func]
    
    H_matrix_func = None
    H_coeffs_real_func = np.zeros(8,dtype=float) 
    t_vals_func_ret = np.linspace(0,1,num_steps_param)
    all_path_coeffs_r_func_ret = np.full((num_steps_param,8), np.nan, dtype=float)

    try:
        if np.allclose(D_disp_func,Id): H_matrix_func = np.zeros((dim,dim),dtype=complex)
        else:
            log_D_f_val=logm(D_disp_func); 
            if np.any(np.isnan(log_D_f_val)) or np.any(np.isinf(log_D_f_val)): raise LinAlgError(f"logm NaN/Inf D{D_eff_tuple_func}")
            H_matrix_func=1j*log_D_f_val
        H_chk_f_val=H_matrix_func.conj().T; 
        if not np.allclose(H_matrix_func,H_chk_f_val,atol=1e-7): H_matrix_func=0.5*(H_matrix_func+H_chk_f_val)
        H_coeffs_cplx_f_val=get_gellmann_coeffs(H_matrix_func,lambdas_dict_param); 
        H_coeffs_real_func=H_coeffs_cplx_f_val.real 
        if np.any(np.abs(H_coeffs_cplx_f_val.imag[~np.isnan(H_coeffs_cplx_f_val.imag)]) > 1e-7):
             pass
    except Exception as e_h_calc_f_final_loc: 
        print(f"Err H_calc D{D_eff_tuple_func} [path {start_pq_tuple}->{end_pq_tuple}]: {e_h_calc_f_final_loc}")
        return t_vals_func_ret, all_path_coeffs_r_func_ret, None, None, D_eff_tuple_func

    for i_f_evol_final, t_f_evol_final in enumerate(t_vals_func_ret): 
        try:
            if H_matrix_func is None or not np.all(np.isfinite(H_matrix_func)): raise ValueError("H_matrix_func invalid for evolution")
            Dt_f_evol_final=expm(-1j*H_matrix_func*t_f_evol_final); 
            if np.any(np.isnan(Dt_f_evol_final)) or np.any(np.isinf(Dt_f_evol_final)): raise LinAlgError("expm NaN/Inf")
            At_f_evol_final=Dt_f_evol_final@A_start_func@Dt_f_evol_final.conj().T; 
            all_path_coeffs_r_func_ret[i_f_evol_final,:]=get_gellmann_coeffs(At_f_evol_final,lambdas_dict_param).real
        except Exception as e_evol_f_final_loc: 
            all_path_coeffs_r_func_ret[i_f_evol_final,:]=np.nan 
            
    if np.any(np.isnan(all_path_coeffs_r_func_ret)): print(f"NaNs present in final path_coeffs for D{D_eff_tuple_func}, path {start_pq_tuple}->{end_pq_tuple}")
    
    return t_vals_func_ret, all_path_coeffs_r_func_ret, H_coeffs_real_func, H_matrix_func, D_eff_tuple_func


def calculate_commutator(A, B): return A @ B - B @ A
def analyze_hamiltonian_symmetries(H, H_coeffs_real, D_eff_tuple_str, tol=1e-9):
    # (Assumed correct)
    print(f"\n  --- Hamiltonian Symmetries (H from {D_eff_tuple_str}) ---")
    h_leak_mags_an_f_final=np.array([np.abs(H_coeffs_real[k-1]) for k in ALL_LEAKAGE_INDICES]) 
    total_h_leak_norm_an_f_final=np.linalg.norm(h_leak_mags_an_f_final); total_h_norm_val_an_f_final=np.linalg.norm(H_coeffs_real) 
    if total_h_norm_val_an_f_final > tol:
        leak_ratio_H_an_f_final=total_h_leak_norm_an_f_final/total_h_norm_val_an_f_final 
        if leak_ratio_H_an_f_final < 0.1: print(f"  H dominantly code-space (leak_ratio_H={leak_ratio_H_an_f_final:.4f}).")
        else: print(f"  H has significant leakage components (leak_ratio_H={leak_ratio_H_an_f_final:.4f}).")
    else: print("  H approx. zero.")
    P_leak_2_an_f_final=np.array([[0,0,0],[0,0,0],[0,0,1]],dtype=complex) 
    if H is not None and not np.allclose(H,np.zeros_like(H),atol=tol):
        norm_comm_H_P_leak_an_f_final=np.linalg.norm(calculate_commutator(H,P_leak_2_an_f_final)) 
        if norm_comm_H_P_leak_an_f_final < tol: print(f"  [H,P_leak_2]~0 (norm={norm_comm_H_P_leak_an_f_final:.2e}). |2> conserved.")
        else: print(f"  [H,P_leak_2]!=0 (norm={norm_comm_H_P_leak_an_f_final:.2e}). |2> can change.")
        if np.allclose(H[0,2],0,atol=tol) and np.allclose(H[2,0],0,atol=tol) and \
           np.allclose(H[1,2],0,atol=tol) and np.allclose(H[2,1],0,atol=tol):
            print("  H approx. block-diag.")
        else: print("  H NOT block-diag.")
    elif H is not None: print("  H effectively zero.")

def classify_hamiltonian_by_su2_leakage(H_coeffs_real, D_eff_tuple_str, min_norm_H_coeff=1e-3, dominance_factor=3.0):
    # (Assumed correct)
    if H_coeffs_real is None or not np.any(np.isfinite(H_coeffs_real)): return f"Invalid H_coeffs for H from {D_eff_tuple_str}", None
    norm_146_cl_f_final=np.linalg.norm(H_coeffs_real[np.array(SU2_146_INDICES)-1]); norm_257_cl_f_final=np.linalg.norm(H_coeffs_real[np.array(SU2_257_INDICES)-1]) 
    norm_diag_cl_f_final=np.linalg.norm(H_coeffs_real[np.array(DIAGONAL_INDICES)-1]); total_norm_h_cl_f_final=np.linalg.norm(H_coeffs_real) 
    if total_norm_h_cl_f_final < min_norm_H_coeff: return f"Zero H (from {D_eff_tuple_str})", None
    is_146dom_cl_f_final=(norm_146_cl_f_final>min_norm_H_coeff and norm_146_cl_f_final>dominance_factor*norm_257_cl_f_final and norm_146_cl_f_final>dominance_factor*norm_diag_cl_f_final) 
    is_257dom_cl_f_final=(norm_257_cl_f_final>min_norm_H_coeff and norm_257_cl_f_final>dominance_factor*norm_146_cl_f_final and norm_257_cl_f_final>dominance_factor*norm_diag_cl_f_final) 
    is_diagdom_cl_f_final=(norm_diag_cl_f_final>min_norm_H_coeff and norm_diag_cl_f_final>dominance_factor*norm_146_cl_f_final and norm_diag_cl_f_final>dominance_factor*norm_257_cl_f_final) 
    class_det_cl_f_final=f"(H from {D_eff_tuple_str})" 
    if is_146dom_cl_f_final: return f"146-dominant {class_det_cl_f_final}", SU2_146_INDICES
    elif is_257dom_cl_f_final: return f"257-dominant {class_det_cl_f_final}", SU2_257_INDICES
    elif is_diagdom_cl_f_final: return f"Diagonal-dominant {class_det_cl_f_final}", DIAGONAL_INDICES
    else:
        if norm_146_cl_f_final > min_norm_H_coeff or norm_257_cl_f_final > min_norm_H_coeff or norm_diag_cl_f_final > min_norm_H_coeff : return f"Mixed {class_det_cl_f_final}", None
        else: return f"Low Magnitude H {class_det_cl_f_final}", None


def plot_3d_combined_leakage_trajectory(t_vals_plot_f_final, all_ak_coeffs_plot_f_final, title_plot_f_final, filename_plot_f_final):
    # (Assumed correct, with SyntaxError fix)
    if all_ak_coeffs_plot_f_final is None or t_vals_plot_f_final is None: print(f"Skipping 3D for {title_plot_f_final} (no data)."); return
    fig_3d_f_final=plt.figure(figsize=(9,8)); ax_3d_f_final=fig_3d_f_final.add_subplot(111,projection='3d')
    coeffs_146_pf_final=all_ak_coeffs_plot_f_final[:,np.array(SU2_146_INDICES)-1]; coeffs_257_pf_final=all_ak_coeffs_plot_f_final[:,np.array(SU2_257_INDICES)-1]
    valid_146_pf_final=~np.any(np.isnan(coeffs_146_pf_final),axis=1); valid_257_pf_final=~np.any(np.isnan(coeffs_257_pf_final),axis=1)
    plotted_flag_3d_f_final=False
    if np.any(valid_146_pf_final):
        ax_3d_f_final.plot(coeffs_146_pf_final[valid_146_pf_final,0],coeffs_146_pf_final[valid_146_pf_final,1],coeffs_146_pf_final[valid_146_pf_final,2],lw=1.5,color=COLOR_146,label=r'($a_1,a_4,a_6$)')
        ax_3d_f_final.scatter(coeffs_146_pf_final[valid_146_pf_final,0][0],coeffs_146_pf_final[valid_146_pf_final,1][0],coeffs_146_pf_final[valid_146_pf_final,2][0],color=COLOR_146,marker='o',s=50,label='Start(146)',depthshade=True,alpha=0.8)
        ax_3d_f_final.scatter(coeffs_146_pf_final[valid_146_pf_final,0][-1],coeffs_146_pf_final[valid_146_pf_final,1][-1],coeffs_146_pf_final[valid_146_pf_final,2][-1],color=COLOR_146,marker='X',s=60,label='End(146)',depthshade=True,alpha=0.8)
        plotted_flag_3d_f_final=True
    if np.any(valid_257_pf_final):
        ax_3d_f_final.plot(coeffs_257_pf_final[valid_257_pf_final,0],coeffs_257_pf_final[valid_257_pf_final,1],coeffs_257_pf_final[valid_257_pf_final,2],lw=1.5,color=COLOR_257,label=r'($a_2,a_5,a_7$)')
        ax_3d_f_final.scatter(coeffs_257_pf_final[valid_257_pf_final,0][0],coeffs_257_pf_final[valid_257_pf_final,1][0],coeffs_257_pf_final[valid_257_pf_final,2][0],color=COLOR_257,marker='o',s=50,label='Start(257)',depthshade=True,alpha=0.8)
        ax_3d_f_final.scatter(coeffs_257_pf_final[valid_257_pf_final,0][-1],coeffs_257_pf_final[valid_257_pf_final,1][-1],coeffs_257_pf_final[valid_257_pf_final,2][-1],color=COLOR_257,marker='X',s=60,label='End(257)',depthshade=True,alpha=0.8)
        plotted_flag_3d_f_final=True
    if not plotted_flag_3d_f_final: print(f"Skipping 3D for {title_plot_f_final} (no valid points)."); plt.close(fig_3d_f_final); return
    ax_3d_f_final.set_xlabel(r'Comp.1 ($a_1,a_2$)',fontsize=9); ax_3d_f_final.set_ylabel(r'Comp.2 ($a_4,a_5$)',fontsize=9); ax_3d_f_final.set_zlabel(r'Comp.3 ($a_6,a_7$)',fontsize=9)
    ax_3d_f_final.tick_params(axis='both',which='major',labelsize=8); ax_3d_f_final.set_title(title_plot_f_final,fontsize=11)
    ax_3d_f_final.legend(fontsize=8,loc='center left',bbox_to_anchor=(1.05,0.5));
    all_x_coords_3df_p_final,all_y_coords_3df_p_final,all_z_coords_3df_p_final = [],[],[] 
    if np.any(valid_146_pf_final): all_x_coords_3df_p_final.append(coeffs_146_pf_final[valid_146_pf_final,0]); all_y_coords_3df_p_final.append(coeffs_146_pf_final[valid_146_pf_final,1]); all_z_coords_3df_p_final.append(coeffs_146_pf_final[valid_146_pf_final,2])
    if np.any(valid_257_pf_final): all_x_coords_3df_p_final.append(coeffs_257_pf_final[valid_257_pf_final,0]); all_y_coords_3df_p_final.append(coeffs_257_pf_final[valid_257_pf_final,1]); all_z_coords_3df_p_final.append(coeffs_257_pf_final[valid_257_pf_final,2])
    if all_x_coords_3df_p_final: 
        all_x_concat_3df_p_final = np.concatenate(all_x_coords_3df_p_final) if all_x_coords_3df_p_final else np.array([])
        all_y_concat_3df_p_final = np.concatenate(all_y_coords_3df_p_final) if all_y_coords_3df_p_final else np.array([])
        all_z_concat_3df_p_final = np.concatenate(all_z_coords_3df_p_final) if all_z_coords_3df_p_final else np.array([])
        if all_x_concat_3df_p_final.size > 0 and all_y_concat_3df_p_final.size > 0 and all_z_concat_3df_p_final.size > 0: 
            min_x_3df_p_final,max_x_3df_p_final=np.min(all_x_concat_3df_p_final),np.max(all_x_concat_3df_p_final) 
            min_y_3df_p_final,max_y_3df_p_final=np.min(all_y_concat_3df_p_final),np.max(all_y_concat_3df_p_final) 
            min_z_3df_p_final,max_z_3df_p_final=np.min(all_z_concat_3df_p_final),np.max(all_z_concat_3df_p_final) 
            ax_3d_f_final.set_xlim([min_x_3df_p_final-0.1*abs(min_x_3df_p_final) if min_x_3df_p_final!=0 else -0.1, max_x_3df_p_final+0.1*abs(max_x_3df_p_final) if max_x_3df_p_final!=0 else 0.1])
            ax_3d_f_final.set_ylim([min_y_3df_p_final-0.1*abs(min_y_3df_p_final) if min_y_3df_p_final!=0 else -0.1, max_y_3df_p_final+0.1*abs(max_y_3df_p_final) if max_y_3df_p_final!=0 else 0.1])
            ax_3d_f_final.set_zlim([min_z_3df_p_final-0.1*abs(min_z_3df_p_final) if min_z_3df_p_final!=0 else -0.1, max_z_3df_p_final+0.1*abs(max_z_3df_p_final) if max_z_3df_p_final!=0 else 0.1])
            range_x_3df_p_final,range_y_3df_p_final,range_z_3df_p_final = max_x_3df_p_final-min_x_3df_p_final,max_y_3df_p_final-min_y_3df_p_final,max_z_3df_p_final-min_z_3df_p_final 
            try: ax_3d_f_final.set_box_aspect((max(0.1,range_x_3df_p_final),max(0.1,range_y_3df_p_final),max(0.1,range_z_3df_p_final)))
            except AttributeError: pass
    plt.tight_layout(rect=[0,0,0.85,1]);
    try:
        fig_3d_f_final.savefig(filename_plot_f_final,dpi=200)
        if GENERATE_FULL_PLOTS: print(f"Saved 3D: {filename_plot_f_final}")
    except Exception as e_s_3d_f_save_final:
        print(f"Error saving 3D plot {filename_plot_f_final}: {e_s_3d_f_save_final}")
    if 'fig_3d_f_final' in locals() and plt.fignum_exists(fig_3d_f_final.number):
        plt.close(fig_3d_f_final)

def get_adjoint_representation_R(D_operator_adj_param_r_final, lambdas_dict_adj_param_r_final): # Renamed parameter
    R_matrix_adj_r_f_final = np.zeros((8, 8), dtype=float)
    lambda_list_adj_r_f_final = [lambdas_dict_adj_param_r_final[k_val_adj_r_f_final_val] for k_val_adj_r_f_final_val in range(1, 9)]
    for j_col_adj_r_f_final_val in range(8):
        lambda_j_adj_r_f_final_val = lambda_list_adj_r_f_final[j_col_adj_r_f_final_val]
        # --- NameError FIX in get_adjoint_representation_R is applied here ---
        D_lambda_j_D_dag_adj_r_f_final_val = D_operator_adj_param_r_final @ lambda_j_adj_r_f_final_val @ D_operator_adj_param_r_final.conj().T
        # --- End FIX ---
        for k_row_adj_r_f_final_val in range(8):
            lambda_k_adj_r_f_final_val = lambda_list_adj_r_f_final[k_row_adj_r_f_final_val]
            coeff_val_adj_r_f_final_val = 0.5 * np.trace(D_lambda_j_D_dag_adj_r_f_final_val @ lambda_k_adj_r_f_final_val)
            R_matrix_adj_r_f_final[k_row_adj_r_f_final_val, j_col_adj_r_f_final_val] = coeff_val_adj_r_f_final_val.real
            if abs(coeff_val_adj_r_f_final_val.imag) > 1e-9: pass
    return R_matrix_adj_r_f_final


# --- Main Loop & Signature Generation ---
num_steps = 100
output_plot_dir = "plots_72_Paths_FullAnalysis_v8_Final" # NEW directory name
if not os.path.exists(output_plot_dir): os.makedirs(output_plot_dir)
print(f"\nPlots will be saved in directory: '{output_plot_dir}/'")

max_leakage_per_D_eff_type = defaultdict(lambda: 0.0)
paths_processed_successfully_count = 0 # Correctly initialized
paths_with_calc_errors_count = 0       # Correctly initialized

path_robust_pattern_words = []
unique_ak_patterns_to_letter = {} 
next_ak_pattern_code = 0
paths_by_robust_word = defaultdict(list)
all_path_grammar_data = [] 

directed_paths_to_plot = list(itertools.permutations(phase_points_tuples, 2))
num_paths_total = len(directed_paths_to_plot)
sampling_indices = np.linspace(0, num_steps - 1, NUM_SAMPLES_PER_AK, dtype=int)

if num_paths_total == 0: print("\nNo paths to plot. Skipping.")
else:
    print(f"\n=== Generating Data for {num_paths_total} DIRECTED Paths ===")
    start_main_loop_time = time.time()
    
    real_leakage_indices_plot = ALL_LEAKAGE_INDICES
    code_space_indices_plot = DIAGONAL_INDICES
    num_ak_cols_for_plot = len(real_leakage_indices_plot)
    debug_rt_paths_done = 0

    for i_path, (start_pq, end_pq) in enumerate(directed_paths_to_plot):
        path_num = i_path + 1
        
        t_vals, all_ak_coeffs, H_coeffs, H_matrix, D_eff_tuple = \
            calculate_path_coeffs_all_lambdas(start_pq, end_pq, lambdas, num_steps) 

        path_id_str = f"Path {start_pq}->{end_pq} (D_eff={D_eff_tuple})" 
        print(f"\n--- Path {path_num}/{num_paths_total}: {path_id_str} ---")

        c_A_start_tuple = None 
        if start_pq in static_A_pq_coeffs:
            c_A_start_tuple = static_A_pq_coeffs[start_pq]
            print(f"  Static Coeffs c(A_start={start_pq}): {c_A_start_tuple}")
        else: print(f"  ERROR: Static coeffs for A_start={start_pq} not found!")

        h_H_eff_tuple = None 
        if H_coeffs is not None:
            h_H_eff_tuple = tuple(np.round(H_coeffs, PATTERN_ROUND_PRECISION))
            print(f"  Coeffs h(H_eff(D{D_eff_tuple})): {h_H_eff_tuple}")
        else: print(f"  H_coeffs for D{D_eff_tuple} is None.")
        
        current_path_ak_letters = [] 
        path_word = "ERR_CALC_PATH_WORD" 
        
        if all_ak_coeffs is not None:
            for k_gm_idx in range(8): 
                ak_series = all_ak_coeffs[:, k_gm_idx] 
                ak_9_sample_tuple = tuple([np.nan] * NUM_SAMPLES_PER_AK) 
                if not (np.all(np.isnan(ak_series)) or len(ak_series) < np.max(sampling_indices) + 1):
                    sampled_ak = ak_series[sampling_indices] 
                    ak_9_sample_tuple = tuple(np.round(sampled_ak, PATTERN_ROUND_PRECISION))
                if ak_9_sample_tuple not in unique_ak_patterns_to_letter:
                    unique_ak_patterns_to_letter[ak_9_sample_tuple] = f"P{next_ak_pattern_code}"
                    next_ak_pattern_code += 1
                current_path_ak_letters.append(unique_ak_patterns_to_letter[ak_9_sample_tuple])
            
            path_word = " ".join(current_path_ak_letters)
            path_robust_pattern_words.append(path_word)
            paths_by_robust_word[path_word].append(path_id_str)
            print(f"  Path Robust Pattern Word: {path_word}")
        else:
            error_word_str_val = "ERR_OP_WORD " * 8; error_word_str_val = error_word_str_val.strip() 
            path_robust_pattern_words.append(error_word_str_val)
            paths_by_robust_word[error_word_str_val].append(f"{path_id_str} (Calc Error)")
            path_word = error_word_str_val 
            print(f"  Path Robust Pattern Word: {path_word}")
            paths_with_calc_errors_count += 1 
            all_path_grammar_data.append({
                'path_num': path_num, 'start_pq': start_pq, 'end_pq': end_pq,
                'D_eff': D_eff_tuple, 
                'c_A_start': c_A_start_tuple if c_A_start_tuple else tuple([np.nan]*8),
                'h_H_eff': h_H_eff_tuple if h_H_eff_tuple else tuple([np.nan]*8), 
                'P_Word': path_word, 'status': 'Calc Error - Evolution'
            })
            continue 
        
        all_path_grammar_data.append({
            'path_num': path_num, 'start_pq': start_pq, 'end_pq': end_pq,
            'D_eff': D_eff_tuple, 'c_A_start': c_A_start_tuple,
            'h_H_eff': h_H_eff_tuple, 'P_Word': path_word, 'status': 'Success'
        })
        paths_processed_successfully_count +=1 

        if all_ak_coeffs is not None and t_vals is not None:
            r_vals_path_curr = np.zeros(num_steps) # Renamed for current scope
            valid_r_calc_path_curr = True       # Renamed for current scope
            if np.all(np.isnan(all_ak_coeffs)):
                r_vals_path_curr[:] = np.nan; valid_r_calc_path_curr = False
                if DEBUG_RT_CALCULATION_MAIN and debug_rt_paths_done < MAX_PATHS_TO_DEBUG_RT_MAIN:
                    print(f"    DEBUG r(t): all_ak_coeffs is all NaN for path {path_num}.")
            else:
                for j_rt_path_curr in range(num_steps): 
                    leakage_coeffs_t_curr = all_ak_coeffs[j_rt_path_curr, np.array(ALL_LEAKAGE_INDICES)-1] 
                    if np.any(np.isnan(leakage_coeffs_t_curr)):
                        r_vals_path_curr[j_rt_path_curr] = np.nan; valid_r_calc_path_curr = False
                    else:
                        r_vals_path_curr[j_rt_path_curr] = np.sqrt(np.sum(leakage_coeffs_t_curr**2))
                        if DEBUG_RT_CALCULATION_MAIN and debug_rt_paths_done < MAX_PATHS_TO_DEBUG_RT_MAIN and j_rt_path_curr % (num_steps//4) == 0 :
                            print(f"    DEBUG r(t) path {path_num}, t_idx {j_rt_path_curr}: leakage_coeffs={np.round(leakage_coeffs_t_curr,3)}, r(t)={r_vals_path_curr[j_rt_path_curr]:.4f}")
            if DEBUG_RT_CALCULATION_MAIN and debug_rt_paths_done < MAX_PATHS_TO_DEBUG_RT_MAIN:
                debug_rt_paths_done +=1
                if r_vals_path_curr[np.isfinite(r_vals_path_curr)].size > 0: 
                    print(f"    DEBUG r(t) path {path_num}: Min r(t)={np.min(r_vals_path_curr[np.isfinite(r_vals_path_curr)]):.4f}, Max r(t)={np.max(r_vals_path_curr[np.isfinite(r_vals_path_curr)]):.4f}")
                else: print(f"    DEBUG r(t) path {path_num}: All r(t) values are NaN or empty after filter.")
            
            if valid_r_calc_path_curr and r_vals_path_curr[np.isfinite(r_vals_path_curr)].size > 0:
                max_r_this_p_val = np.max(r_vals_path_curr[np.isfinite(r_vals_path_curr)]) 
                max_leakage_per_D_eff_type[D_eff_tuple] = \
                    max(max_leakage_per_D_eff_type.get(D_eff_tuple, 0.0), max_r_this_p_val if np.isfinite(max_r_this_p_val) else 0.0)
            elif valid_r_calc_path_curr :
                 max_leakage_per_D_eff_type[D_eff_tuple] = \
                    max(max_leakage_per_D_eff_type.get(D_eff_tuple, 0.0), 0.0)
        
        if GENERATE_FULL_PLOTS:
            # (Full 2D plotting)
            fig_2d_main_plots = plt.figure(figsize=(12,10)); gs_main_plots=fig_2d_main_plots.add_gridspec(3,num_ak_cols_for_plot,height_ratios=[2.5,1,1],wspace=0.3,hspace=0.5)
            ax_env_main_plots = fig_2d_main_plots.add_subplot(gs_main_plots[0,:]) 
            ax_env_main_plots.plot(t_vals,r_vals_path_curr,'-',linewidth=2.5,color='black',label='Total Leakage ($r(t)$)',zorder=5) # Use r_vals_path_curr
            for k_env_main_plots in real_leakage_indices_plot: ax_env_main_plots.plot(t_vals,np.abs(all_ak_coeffs[:,k_env_main_plots-1]),'--',linewidth=1.2,color=leakage_channel_colors[k_env_main_plots],label=f'$|Re({lambda_names[k_env_main_plots]})|$',alpha=0.7)
            ax_env_main_plots.set_title(path_id_str+' (Leakage Envelope)',fontsize=10,pad=10)
            ax_env_main_plots.set_xlabel('Time ($t$)',fontsize=8); ax_env_main_plots.set_ylabel('Amplitude',fontsize=8)
            ax_env_main_plots.tick_params(axis='both',which='major',labelsize=7); ax_env_main_plots.grid(True,linestyle=':',alpha=0.7)
            ax_env_main_plots.legend(loc='center left',bbox_to_anchor=(1.01,0.5),fontsize=7)
            max_r_env_plot_main_plots = 0.1 
            if r_vals_path_curr[np.isfinite(r_vals_path_curr)].size>0: max_r_env_plot_main_plots = np.max(r_vals_path_curr[np.isfinite(r_vals_path_curr)])
            ax_env_main_plots.set_ylim(0,max(0.1,max_r_env_plot_main_plots*1.1))
            all_ak_axes_main_plots = [] 
            for col_main_plots, k_main_leak_plots in enumerate(real_leakage_indices_plot): 
                ax_re_main_plots = fig_2d_main_plots.add_subplot(gs_main_plots[1,col_main_plots]) 
                ax_re_main_plots.plot(t_vals,all_ak_coeffs[:,k_main_leak_plots-1],color=leakage_channel_colors[k_main_leak_plots],linewidth=1.5)
                ax_re_main_plots.set_title(f'Re({lambda_names[k_main_leak_plots]})',fontsize=7,pad=2)
                ax_re_main_plots.tick_params(axis='both',which='major',labelsize=6,pad=0); ax_re_main_plots.set_xticks([0,0.5,1]); ax_re_main_plots.set_xticklabels(['0','0.5','1']); ax_re_main_plots.grid(True,linestyle=':',alpha=0.6)
                if col_main_plots==0: ax_re_main_plots.set_ylabel('Amplitude',fontsize=7,labelpad=0)
                all_ak_axes_main_plots.append(ax_re_main_plots)
            for col_main_c_plots, k_main_code_plots in enumerate(code_space_indices_plot): 
                ax_code_main_plots = fig_2d_main_plots.add_subplot(gs_main_plots[2,col_main_c_plots]) 
                ax_code_main_plots.plot(t_vals,all_ak_coeffs[:,k_main_code_plots-1],color=code_space_color,linewidth=1.5)
                ax_code_main_plots.set_title(f'Re({lambda_names[k_main_code_plots]}) (Code)',fontsize=7,pad=2)
                ax_code_main_plots.tick_params(axis='both',which='major',labelsize=6,pad=0); ax_code_main_plots.set_xticks([0,0.5,1]); ax_code_main_plots.set_xticklabels(['0','0.5','1']); ax_code_main_plots.grid(True,linestyle=':',alpha=0.6)
                if col_main_c_plots==0: ax_code_main_plots.set_ylabel('Amplitude',fontsize=7,labelpad=0)
                all_ak_axes_main_plots.append(ax_code_main_plots)
            for col_main_fill_plots in range(len(code_space_indices_plot),num_ak_cols_for_plot): 
                fig_2d_main_plots.add_subplot(gs_main_plots[2,col_main_fill_plots]).set_visible(False)
            ak_vals_ylim_main_plots = all_ak_coeffs[:,np.array(real_leakage_indices_plot+code_space_indices_plot)-1] 
            finite_ak_ylim_main_plots = ak_vals_ylim_main_plots[np.isfinite(ak_vals_ylim_main_plots)] 
            max_ak_abs_main_plots = np.max(np.abs(finite_ak_ylim_main_plots)) if len(finite_ak_ylim_main_plots)>0 else 0.1 
            ak_subplot_ylim_main_plots = (-max(0.1,max_ak_abs_main_plots*1.1),max(0.1,max_ak_abs_main_plots*1.1)) 
            for ax_s_main_plots in all_ak_axes_main_plots: ax_s_main_plots.set_ylim(ak_subplot_ylim_main_plots) 
            fig_2d_main_plots.suptitle(path_id_str + " - 2D Analysis", fontsize=14)
            plt.tight_layout(rect=[0,0.03,1,0.96])
            fname_2d_main_plots_png = os.path.join(output_plot_dir, f"path_{path_num:02d}_{start_pq[0]}{start_pq[1]}to{end_pq[0]}{end_pq[1]}_2D.png")
            try: fig_2d_main_plots.savefig(fname_2d_main_plots_png,dpi=300,bbox_inches='tight'); print(f"Saved 2D: {fname_2d_main_plots_png}")
            except Exception as e_s_2d_main_plots: print(f"Err save 2D path {path_num}: {e_s_2d_main_plots}")
            plt.close(fig_2d_main_plots)
            
            H_classification_str_plot_main_plots, _ = classify_hamiltonian_by_su2_leakage(H_coeffs, f"D{D_eff_tuple}")
            title_3d_plot_main_plots = f"Comb. Leakage ({H_classification_str_plot_main_plots.split('(')[0].strip()})\n{path_id_str}"
            fname_3d_plot_main_plots_png = os.path.join(output_plot_dir, f"path_{path_num:02d}_{start_pq[0]}{start_pq[1]}to{end_pq[0]}{end_pq[1]}_3D.png")
            plot_3d_combined_leakage_trajectory(t_vals, all_ak_coeffs, title_3d_plot_main_plots, fname_3d_plot_main_plots_png)


    # --- Post-Loop Analyses ---
    # 1. "Robust Pattern Word" Uniqueness & Grouping
    print("\n\n=== Path 'Robust Pattern Word' Signature Uniqueness & Grouping ===")
    if path_robust_pattern_words:
        robust_word_counts_post_final = Counter(path_robust_pattern_words)
        num_unique_robust_words_post_final = len(robust_word_counts_post_final)
        print(f"Found {num_unique_robust_words_post_final} unique 8-part 'robust pattern words' out of {paths_processed_successfully_count} successfully processed paths.")
        print(f"  (Each part 'P#' corresponds to a unique {NUM_SAMPLES_PER_AK}-sample numerical pattern for an a_k(t))")
        if num_unique_robust_words_post_final == paths_processed_successfully_count and paths_processed_successfully_count > 0: 
            print("All successfully processed path 'robust pattern words' are unique.")
        elif paths_processed_successfully_count > 0 :
            print(f"{paths_processed_successfully_count - num_unique_robust_words_post_final} successfully processed paths share 'robust pattern words'.")
        print(f"\nTotal unique {NUM_SAMPLES_PER_AK}-sample a_k(t) patterns found (P0 to P{next_ak_pattern_code-1}): {next_ak_pattern_code}")
        print("\n--- Grouping of Paths by Robust Pattern Word (Sorted by Freq) ---")
        sorted_words_post_grp_final_val = sorted(paths_by_robust_word.items(), key=lambda item_grp_post_final_val: (len(item_grp_post_final_val[1]), item_grp_post_final_val[0]), reverse=True) 
        for robust_word_key_grp_final_val, path_list_grp_final_val in sorted_words_post_grp_final_val:
            print(f"\nRobust Word: '{robust_word_key_grp_final_val}' (Generated by {len(path_list_grp_final_val)} path(s)):")
            for path_desc_grp_final_val in path_list_grp_final_val:
                print(f"  - {path_desc_grp_final_val}")
    else: print("No 'robust pattern word' signatures were generated.")

    # 2. Adjoint Representation R(D) analysis
    print("\n\n=== Verifying D A D_dag = A_transformed via Adjoint Rep. R(D) ===")
    static_A_pq_coeffs_np_adj_post_final_val = {k_adj_np_f_val_run_adj: np.array(v_adj_np_f_val_run_adj) for k_adj_np_f_val_run_adj, v_adj_np_f_val_run_adj in static_A_pq_coeffs.items()} 
    for D_eff_tuple_for_adj_post_final_val, D_operator_for_adj_post_final_val in D_ops_num.items(): 
        if D_eff_tuple_for_adj_post_final_val == (0,0): continue
        print(f"\n--- For Displacement Operator D{D_eff_tuple_for_adj_post_final_val} ---")
        R_D_for_adj_post_final_val = get_adjoint_representation_R(D_operator_for_adj_post_final_val, lambdas) 
        identity_8x8_for_adj_post_final_val = np.identity(8) 
        is_orthogonal_for_adj_post_final_val = np.allclose(R_D_for_adj_post_final_val.T @ R_D_for_adj_post_final_val, identity_8x8_for_adj_post_final_val, atol=1e-5)
        det_R_D_for_adj_post_final_val = np.linalg.det(R_D_for_adj_post_final_val) 
        print(f"  R(D{D_eff_tuple_for_adj_post_final_val}) is orthogonal: {is_orthogonal_for_adj_post_final_val} (||R^T R - I||_F = {np.linalg.norm(R_D_for_adj_post_final_val.T @ R_D_for_adj_post_final_val - identity_8x8_for_adj_post_final_val):.2e})")
        print(f"  Determinant of R(D{D_eff_tuple_for_adj_post_final_val}): {det_R_D_for_adj_post_final_val:.4f}")
        consistent_transformations_for_adj_post_final_val = 0 
        for start_pq_for_adj_post_final_val, c_start_vec_for_adj_post_final_val in static_A_pq_coeffs_np_adj_post_final_val.items(): 
            p_transformed_for_adj_post_final_val = (D_eff_tuple_for_adj_post_final_val[0] + start_pq_for_adj_post_final_val[0]) % dim 
            q_transformed_for_adj_post_final_val = (D_eff_tuple_for_adj_post_final_val[1] + start_pq_for_adj_post_final_val[1]) % dim 
            A_transformed_pq_tuple_for_adj_post_final_val = (p_transformed_for_adj_post_final_val, q_transformed_for_adj_post_final_val) 
            c_expected_end_for_adj_post_final_val = static_A_pq_coeffs_np_adj_post_final_val[A_transformed_pq_tuple_for_adj_post_final_val] 
            c_calculated_end_for_adj_post_final_val = R_D_for_adj_post_final_val @ c_start_vec_for_adj_post_final_val 
            if np.allclose(c_calculated_end_for_adj_post_final_val, c_expected_end_for_adj_post_final_val, atol=10**-(PATTERN_ROUND_PRECISION-1) ): 
                consistent_transformations_for_adj_post_final_val += 1
            else:
                print(f"    MISMATCH for D{D_eff_tuple_for_adj_post_final_val} acting on A{start_pq_for_adj_post_final_val} -> A{A_transformed_pq_tuple_for_adj_post_final_val}:")
                diff_vec_for_adj_post_final_val = c_calculated_end_for_adj_post_final_val - c_expected_end_for_adj_post_final_val 
                print(f"      Max Diff Element          : {np.max(np.abs(diff_vec_for_adj_post_final_val)):.2e}")
        print(f"  D{D_eff_tuple_for_adj_post_final_val} consistently transformed {consistent_transformations_for_adj_post_final_val}/9 static A_pq coefficient vectors via R(D).")

    # 3. Super-Symbol Analysis
    print("\n\n=== Analysis of P# Pattern Symmetries (Super-Symbols) ===")
    super_symbol_map_ss_final = {} 
    super_symbol_canonical_members_ss_final = {} 
    next_ss_code_final = 0 
    processed_p_labels_for_ss_final = set() 
    if unique_ak_patterns_to_letter:
        letter_to_pattern_tuples_ss_final = {v_ss_f: k_ss_f for k_ss_f, v_ss_f in unique_ak_patterns_to_letter.items()} 
        sorted_p_labels_ss_final = sorted(letter_to_pattern_tuples_ss_final.keys(), key=lambda x_ss_f_sort: int(x_ss_f_sort[1:])) 
        for p_label_A_ss_f in sorted_p_labels_ss_final: 
            if p_label_A_ss_f in processed_p_labels_for_ss_final: continue
            pattern_A_tuple_ss_f = letter_to_pattern_tuples_ss_final[p_label_A_ss_f]
            family_candidates_ss_f_list = [{'tuple':pattern_A_tuple_ss_f, 'relation_to_A':'original', 'P_label':p_label_A_ss_f}]
            inv_samples_ss_f = tuple(np.round([-s_ss_f if np.isfinite(s_ss_f) else np.nan for s_ss_f in pattern_A_tuple_ss_f], PATTERN_ROUND_PRECISION))
            if inv_samples_ss_f in unique_ak_patterns_to_letter: family_candidates_ss_f_list.append({'tuple':inv_samples_ss_f,'relation_to_A':'inverted','P_label':unique_ak_patterns_to_letter[inv_samples_ss_f]})
            tr_samples_ss_f = tuple(np.round(np.array(pattern_A_tuple_ss_f[::-1]), PATTERN_ROUND_PRECISION))
            if tr_samples_ss_f in unique_ak_patterns_to_letter: family_candidates_ss_f_list.append({'tuple':tr_samples_ss_f,'relation_to_A':'time-reversed','P_label':unique_ak_patterns_to_letter[tr_samples_ss_f]})
            inv_tr_samples_ss_f = tuple(np.round(np.array([-s_ss_f if np.isfinite(s_ss_f) else np.nan for s_ss_f in pattern_A_tuple_ss_f[::-1]]), PATTERN_ROUND_PRECISION))
            if inv_tr_samples_ss_f in unique_ak_patterns_to_letter: family_candidates_ss_f_list.append({'tuple':inv_tr_samples_ss_f,'relation_to_A':'inverted-time-reversed','P_label':unique_ak_patterns_to_letter[inv_tr_samples_ss_f]})
            current_family_p_labels_set_ss_f = set(); deduplicated_family_members_info_ss_f = []
            for cand_ss_f in family_candidates_ss_f_list:
                if cand_ss_f['P_label'] not in current_family_p_labels_set_ss_f:
                    current_family_p_labels_set_ss_f.add(cand_ss_f['P_label']); deduplicated_family_members_info_ss_f.append(cand_ss_f)
            if not deduplicated_family_members_info_ss_f: continue
            canonical_p_label_ss_f = min(deduplicated_family_members_info_ss_f, key=lambda x_ss_min_f: int(x_ss_min_f['P_label'][1:]))['P_label']
            canonical_pattern_tuple_ss_f = letter_to_pattern_tuples_ss_final[canonical_p_label_ss_f]
            ss_label_for_family_val_f = "" 
            if canonical_p_label_ss_f in super_symbol_map_ss_final: ss_label_for_family_val_f = super_symbol_map_ss_final[canonical_p_label_ss_f]
            else: 
                ss_label_for_family_val_f = f"S{next_ss_code_final}"; next_ss_code_final += 1
                super_symbol_canonical_members_ss_final[ss_label_for_family_val_f] = {'canonical_P':canonical_p_label_ss_f, 'members':{}}
            for member_info_ss_f in deduplicated_family_members_info_ss_f:
                member_p_label_ss_f = member_info_ss_f['P_label']; member_pattern_tuple_ss_f = member_info_ss_f['tuple']
                super_symbol_map_ss_final[member_p_label_ss_f] = ss_label_for_family_val_f; processed_p_labels_for_ss_final.add(member_p_label_ss_f)
                rel_to_canonical_ss_f = "unknown"
                if member_pattern_tuple_ss_f == canonical_pattern_tuple_ss_f: rel_to_canonical_ss_f = "canonical"
                elif member_pattern_tuple_ss_f == tuple(np.round([-s_can_f if np.isfinite(s_can_f) else np.nan for s_can_f in canonical_pattern_tuple_ss_f], PATTERN_ROUND_PRECISION)): rel_to_canonical_ss_f = "inverted"
                elif member_pattern_tuple_ss_f == tuple(np.round(np.array(canonical_pattern_tuple_ss_f[::-1]), PATTERN_ROUND_PRECISION)): rel_to_canonical_ss_f = "time-reversed"
                elif member_pattern_tuple_ss_f == tuple(np.round(np.array([-s_can_f if np.isfinite(s_can_f) else np.nan for s_can_f in canonical_pattern_tuple_ss_f[::-1]]), PATTERN_ROUND_PRECISION)): rel_to_canonical_ss_f = "inverted-time-reversed"
                super_symbol_canonical_members_ss_final[ss_label_for_family_val_f]['members'][member_p_label_ss_f] = rel_to_canonical_ss_f
        print(f"\n--- Identified {next_ss_code_final} Super-Symbol Families ---")
        for ss_id_p_f_val, details_p_f_val in sorted(super_symbol_canonical_members_ss_final.items(), key=lambda x_p_sort_f_key: int(x_p_sort_f_key[0][1:])): 
            print(f"\nSuper-Symbol: {ss_id_p_f_val} (Canonical P-Pattern: {details_p_f_val['canonical_P']})")
            canonical_p_for_print_final_val = details_p_f_val['canonical_P']
            if canonical_p_for_print_final_val in letter_to_pattern_tuples_ss_final:
                 print(f"  Canonical Pattern Samples: {np.round(letter_to_pattern_tuples_ss_final[canonical_p_for_print_final_val], 3)}")
            else: print(f"  Canonical Pattern Samples for {canonical_p_for_print_final_val}: Data not found.")
            print(f"  Members:")
            for member_p_s_f_val, rel_p_s_f_val in sorted(details_p_f_val['members'].items(), key=lambda x_p_s_sort_f_key: int(x_p_s_sort_f_key[0][1:])): 
                print(f"    - {member_p_s_f_val}: {rel_p_s_f_val} of canonical {details_p_f_val['canonical_P']}")
    else: print("No unique a_k patterns map for Super-Symbol analysis.")

    # 4. Summary Plot (Leakage)
    if True: 
        print("\n=== Generating Summary Plot: Max Leakage per Effective D(m,n) Type ===")
        fig_summary_final_run_fix_nameerr, ax_summary_final_run_fix_nameerr = plt.subplots(figsize=(8,6)) 
        dp_coords_final_run_fix_nameerr, dq_coords_final_run_fix_nameerr = np.arange(dim),np.arange(dim)
        max_leakage_grid_final_run_fix_nameerr = np.zeros((dim,dim))
        for (dp_s_script_final_fix_nameerr,dq_s_script_final_fix_nameerr),max_r_s_val_script_final_fix_nameerr in max_leakage_per_D_eff_type.items(): max_leakage_grid_final_run_fix_nameerr[dp_s_script_final_fix_nameerr,dq_s_script_final_fix_nameerr]=max_r_s_val_script_final_fix_nameerr 
        cax_s_script_final_fix_nameerr = ax_summary_final_run_fix_nameerr.imshow(max_leakage_grid_final_run_fix_nameerr,cmap='viridis',origin='lower',extent=[-0.5,dim-0.5,-0.5,dim-0.5],aspect='equal')
        for m_s_script_final_fix_nameerr in dp_coords_final_run_fix_nameerr:
            for n_s_script_final_fix_nameerr in dq_coords_final_run_fix_nameerr:
                leak_val_s_script_final_fix_nameerr = max_leakage_grid_final_run_fix_nameerr[m_s_script_final_fix_nameerr,n_s_script_final_fix_nameerr] 
                text_col_s_script_final_fix_nameerr = 'white' if leak_val_s_script_final_fix_nameerr < (np.max(max_leakage_grid_final_run_fix_nameerr)*0.6 if np.max(max_leakage_grid_final_run_fix_nameerr)>0 else 0.1) else 'black'
                ax_summary_final_run_fix_nameerr.text(n_s_script_final_fix_nameerr,m_s_script_final_fix_nameerr,f'{leak_val_s_script_final_fix_nameerr:.4f}',va='center',ha='center',color=text_col_s_script_final_fix_nameerr,fontsize=9) 
        ax_summary_final_run_fix_nameerr.set_xticks(dq_coords_final_run_fix_nameerr); ax_summary_final_run_fix_nameerr.set_xticklabels([str(n) for n in dq_coords_final_run_fix_nameerr])
        ax_summary_final_run_fix_nameerr.set_yticks(dp_coords_final_run_fix_nameerr); ax_summary_final_run_fix_nameerr.set_yticklabels([str(m) for m in dp_coords_final_run_fix_nameerr])
        ax_summary_final_run_fix_nameerr.set_xlabel('n (for $D_{eff}(m,n)$)',fontsize=12); ax_summary_final_run_fix_nameerr.set_ylabel('m (for $D_{eff}(m,n)$)',fontsize=12)
        title_str_s_final_fix_nameerr = f'Max Leakage $r(t)$ per $D_{{eff}}$ (across {num_paths_total} paths)' # NameError FIX
        ax_summary_final_run_fix_nameerr.set_title(title_str_s_final_fix_nameerr, fontsize=14)
        cbar_s_script_final_fix_nameerr = fig_summary_final_run_fix_nameerr.colorbar(cax_s_script_final_fix_nameerr); cbar_s_script_final_fix_nameerr.set_label('Max Leakage Radius $r(t)$',fontsize=10)
        sum_png_fname_s_final_fix_nameerr = os.path.join(output_plot_dir,"summary_max_leakage_per_Deff_72paths_FullAnalysis_FIXED_vFinal.png") 
        try: fig_summary_final_run_fix_nameerr.savefig(sum_png_fname_s_final_fix_nameerr,dpi=300,bbox_inches='tight'); print(f"Saved: {sum_png_fname_s_final_fix_nameerr}")
        except Exception as e_sss_s_final_loc_fix_nameerr: print(f"Error saving summary plot: {e_sss_s_final_loc_fix_nameerr}")
        plt.close(fig_summary_final_run_fix_nameerr)


    end_main_loop_final_time_val_fix_nameerr_val = time.time()
    print(f"\nProcessed {num_paths_total} paths in {end_main_loop_final_time_val_fix_nameerr_val-start_main_loop_time:.2f} sec.")
    if GENERATE_FULL_PLOTS:
        print(f"  ({paths_processed_successfully_count} full plots generated for successful paths, {paths_with_calc_errors_count} path calculations failed).")
    else: 
        print(f"  ({paths_processed_successfully_count} paths had robust pattern words computed, {paths_with_calc_errors_count} path calculations failed).")

total_script_time_end_fix_nameerr_final_val = time.time() - start_time
print(f"\n=== Total script execution time: {total_script_time_end_fix_nameerr_final_val:.2f} seconds ===")

Script started at: Tue May 27 12:47:23 2025
GENERATE_FULL_PLOTS: False
NUM_SAMPLES_PER_AK for P-label pattern: 9
PATTERN_ROUND_PRECISION: 6

--- Populating Operators ---
Done populating A(p,q) and D(m,n).


=== Verifying Operator Identity D A D_dag = A_transformed (Matrix Level) ===
Operator Identity Check Complete: 81/81 transformations verified.

--- Pre-calculating Static Gell-Mann Coefficients for A_pq operators ---
Done pre-calculating static coefficients for 9 A_pq operators.

Plots will be saved in directory: 'plots_72_Paths_FullAnalysis_v8_Final/'

=== Generating Data for 72 DIRECTED Paths ===

--- Path 1/72: Path (0, 0)->(0, 1) (D_eff=(0, 1)) ---
  Static Coeffs c(A_start=(0, 0)): (0.0, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0, 0.288675)
  Coeffs h(H_eff(D(0, 1))): (-0.0, 1.2092, -0.0, 0.0, -1.2092, 0.0, 1.2092, 0.0)
  Path Robust Pattern Word: P0 P1 P2 P3 P1 P4 P1 P5
    DEBUG r(t) path 1, t_idx 0: leakage_coeffs=[0. 0. 0. 0. 1. 0.], r(t)=1.0000
    DEBUG r(t) path 1, t_idx 25: leakage_c